<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Pre-trained-LM-selection-and-training/Hyper_parameter_search_and_class_imbalance_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyper-parameter search Research

Hyper-parameter : https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/

Transformer hyper-parameter search: https://huggingface.co/blog/ray-tune

**What is hyper-parameter**?
  * Parameters that are used to control the learning process of a model
  * "Model configuration parameters set by the developer to guide learning process for specific dataset".

**Difference between model parameters and model hyper-parameters**?
  * Model parameters: 
    * Variables whose values are not set but learned during the training of a model for specific data.
      * E.g. 
        * Weights (importance given to each feature of an instance) and biases (adjust the generalization of the model) in NN
        * Support vectors in SVM
        * Coefficients in regression models 
  * Model Hyper-perameter:
    * Configuration variable set before training to improve the training process or reduce the loss function.
    * E.g.
      * Learning rate for NN
      * K in KNN

**Hyper-parameter search/tuning/optimization:**
  * No rule of thumb to set hyper parameters and it is required to search for best hyper-parameters of a model on a dataset.
  * Hyper-parameter for a model is searched in search space where each dimention represents hyper-parameter and point represent one model configuration.
  * Goal of hyper-parameter search is to find an optimal configuration parameters (vector) from search space.
  * Different algorithms
    * Random search: randomly sample points from bounded domain of search space
      * More time to search 
      *`RandomizedSearchCV(model,space)` from sklearn, space is a dictionary of parameters to be searched
    * Grid search:  Search space as grid of hyper-parameters and evaluate every
 point in the grid.
      * More defined search in the search space
      * `GridSearchCV(model,space)` from sklearn, space is a dictionary of parameters to be searched.
    * Advanced:
      * Bayesian optimization 
      * Population based training


**Transformers Hyper-parameter tuning :**

Library : RayTune (python library for experiment execution and hyperparameter tuning)

Steps:
  1. Define search space
      * BERT Model fine-tune Hyper-parameters(baseline : https://www.aclweb.org/anthology/N19-1423/):
        * Batch_size : [16,32]
        * Learning rate (adam) : 5e-5,3e-5,2e-5
        * Number of epochs : 2,3,4
      * RoBERTa Model fine-tune hyper-parameters in paper(baseline : https://arxiv.org/abs/1907.11692):
        * Batch_size : [16,32]
        * Learning rate (adam) : 1e-5,2e-5,3e-5
        * Max number of epochs (adam) : 10
        * Weight decay : 0.1
        * Learning rate decay : Linear
        * Warmup ratio : 0.06 
      * GPT-2 Model fine-tune hyper-parameters in paper(baseline : http://www.persagen.com/files/misc/radford2019language.pdf):
        * Auto-regressive model
      * XLNet-large fine-tune Model hyper-parameters in paper(baseline : https://arxiv.org/pdf/1906.08237.pdf):
        * Same as BERT 
        * Batch_size : [16,32]
        * Learning rate (adam) : 5e-5,3e-5,2e-5
        * Number of epochs : 2,3,4
  2. Load Model tokenizer
  3. Load training and evaluation dataset
  4. Define metrics to be evaluated 
    * `Datasets` library from transformers contain metrics which can be used 
    * https://huggingface.co/metrics
  5. Encode training examples
  6. Initialize model 
    * `AutoModelForSequenceClassification.from_pretrained('bert-base-cased', return_dict=True)`
  7. Define `trainer` from transformers
    * Trainer classes provide feature complete API
    * Before instantiating trainer, training arguments should be created to access customization during training
    * https://huggingface.co/transformers/main_classes/trainer.html





Hugging-face Multi-label classification 

* Link : https://colab.research.google.com/drive/18vy67le2DC-iMJK-AiB0vVKtMRAxmBnB?usp=sharing
* Link : https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=Ytdiy3hJJ88P

# Data-preprocessing

In [1]:
! pip install optuna --quiet
! pip install ray[tune] --quiet
# !pip install transformers --quiet

     |████████████████████████████████| 301 kB 27.6 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 164 kB 65.8 MB/s 
     |████████████████████████████████| 75 kB 5.6 MB/s 
     |████████████████████████████████| 111 kB 76.2 MB/s 
     |████████████████████████████████| 141 kB 67.6 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
     |████████████████████████████████| 51.6 MB 38 kB/s 
     |████████████████████████████████| 10.1 MB 71.6 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 3.1 MB 56.4 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 72 kB 520 kB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 
     |████████████████████████████████| 1.3 MB 68.6 MB/s 
     |████████████████████████████████| 124 kB 74.8 MB/s 
     |████████████████████████████████| 294 kB 66.4 MB/s 
     |████████████████████

In [2]:
pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 636 kB 70.4 MB/s 
     |████████████████████████████████| 895 kB 64.6 MB/s 


In [3]:
pip install "ray[tune]" transformers datasets --quiet

     |████████████████████████████████| 542 kB 29.6 MB/s 
     |████████████████████████████████| 243 kB 69.7 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |████████████████████████████████| 118 kB 56.9 MB/s 


In [4]:
!nvidia-smi

Sun Jul 25 09:14:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments

# specify GPU
device = torch.device("cuda")

In [6]:
print(f"Running on transformers v{transformers.__version__}")

Running on transformers v4.10.0.dev0


In [7]:
MAX_LEN = 50
RANDOM_SEED = 47
# pl.seed_everything(RANDOM_SEED)

In [8]:
df = pd.read_csv("/content/ohe_multilabel.csv", index_col = 0)

In [9]:
df

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Many people live in Ethiopia. The people are v...
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,My professor is a hispanic man He came here il...
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,The schoolgirl is walking down the street. She...
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,We'll likely fly over Cape Verde on our flight...
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,I know many people who are Russian. They love ...
...,...,...,...,...,...,...,...,...
16539,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Arab people are associated with hi.
16540,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Car was a new chemist in our lab.
16541,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"The gentlemen stood around the fire, carefully..."
16542,0.0,0.0,0.0,0.0,0.0,0.0,1.0,The common ukrainian is a supporter of a floun...


Stratified sampling :

* Why?
  * With very small or very imbalanced data sets, it's quite possible that the random split could completely eliminate a class from one of the train/test splits.
  * hence, setting `stratify = dependent_variable` makes sure that train and test splits have the same proportion of sampling. 

In [10]:
y = df.iloc[:,:-1].values
X = df.iloc[:,-1].values

In [11]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [12]:
train_df_text

array(['The grandfather dined at the restaurant. He enjoyed salsa dancing and kung fu.',
       'Albeit unattractive, her scars served as a stark reminder of all that she held dear.',
       'there is a new middle eastern ice cream company that has gotten explosive response from critics it s called ice is',
       ..., 'Yemen is near away from california',
       'The shirt is too wet to wear.',
       "The telephone's constant ringing is annoying."], dtype=object)

In [13]:
LABEL_COLUMN = ['Ethnicity',	'gender'	,'profession'	,'religion',	'Anti-stereotype',	'stereotype',	'unrelated']

In [14]:
train_df_labels = pd.DataFrame(train_df_labels, columns= LABEL_COLUMN)
val_df_labels = pd.DataFrame(val_df_labels, columns= LABEL_COLUMN)
test_df_labels = pd.DataFrame(test_df_labels, columns= LABEL_COLUMN)
train_df_text = pd.DataFrame(train_df_text, columns = ['sentence'])
val_df_text = pd.DataFrame(val_df_text, columns = ['sentence'])
test_df_text = pd.DataFrame(test_df_text, columns = ['sentence'])

In [15]:
train_df = pd.concat([train_df_text,train_df_labels], axis = 1)
val_df = pd.concat([val_df_text,val_df_labels], axis = 1)
test_df = pd.concat([test_df_text,test_df_labels], axis = 1)

In [16]:
train_df

,sentence,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated
0,The grandfather dined at the restaurant. He en...,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,"Albeit unattractive, her scars served as a sta...",0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,there is a new middle eastern ice cream compan...,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,while they are in prison the kikes should get ...,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,The Historian was a teacher He loved history a...,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
11575,Most butchers are window.,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11576,Muslim households are very polyester places.,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11577,Yemen is near away from california,1.0,0.0,0.0,0.0,1.0,0.0,0.0
11578,The shirt is too wet to wear.,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
train_df.iloc[:,1:].values

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [18]:
from torch.utils.data import Dataset, DataLoader

In [19]:
class ExplicitStereotypeDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer,max_token_len: int = 50):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index: int):
    data_row = self.data.iloc[0]
    text = data_row[0]
    labels = data_row[1:]
 

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      attention_mask=encoding["attention_mask"].flatten(),
      input_ids=encoding["input_ids"].flatten(),
      labels= torch.FloatTensor(labels)
    )

In [ ]:
sample = train_dataset[0]

In [ ]:
sample

In [20]:
# num_labels = len(sample['labels'])
num_labels = 7

In [21]:
def my_hp_space(trial):
    from ray import tune

    return {
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "num_train_epochs": tune.choice([2,3,5]),
        "seed": tune.choice(range(1, 41)),
        "per_device_train_batch_size": tune.choice([8, 16, 32]),
    }

## BERT

In [ ]:
model = 'bert-base-uncased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model,problem_type="multi_label_classification")

In [ ]:
train_dataset = ExplicitStereotypeDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [ ]:
val_dataset = ExplicitStereotypeDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model, problem_type="multi_label_classification", num_labels = num_labels )

In [ ]:
# from pytorch_lightning.metrics.functional import accuracy, f1, auroc

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     roc_auc = auroc(predictions, labels)
#     return roc_auc

In [ ]:
# Evaluate during training and a bit more often than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.

trainer = Trainer(
    model_init= model_init,
    tokenizer = tokenizer,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

"The default objective to maximize/minimize when doing an hyperparameter search. It is the evaluation loss if no
    metrics are provided to the :class:`~transformers.Trainer`, the sum of all metrics otherwise."

Optuna : By default for hp_search

Metrics :

```
def default_hp_space_ray(trial) -> Dict[str, float]:
    from .integrations import is_ray_tune_available

    assert is_ray_tune_available(), "This function needs ray installed: `pip " "install ray[tune]`"
    from ray import tune

    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "num_train_epochs": tune.choice(list(range(1, 6))),
        "seed": tune.uniform(1, 40),
        "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
    }
```
Link : https://huggingface.co/transformers/_modules/transformers/trainer_utils.html

In [ ]:
# Defaut objective is the sum of all metrics when metrics are provided, so we have to maximize it.
trainer.hyperparameter_search(n_trials=3, hp_space=my_hp_space, backend='ray')

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
2021-07-13 12:33:57,526	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


== Status ==
Memory usage on this node: 2.7/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-07-13_12-34-01
Number of trials: 3/3 (3 PENDING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------|
| _objective_9a82c_00000 | PENDING  |       |     2.49816e-05 |                  2 |                            32 |
| _objective_9a82c_00001 | PENDING  |       |     3.92798e-05 |                  2 |                             8 |
| _objective_9a82c_00002 | PENDING  |       |     1.62407e-05 |                  5 |                            32 |
+--

(pid=359) 2021-07-13 12:34:03.895239: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Memory usage on this node: 3.3/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-07-13_12-34-01
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------|
| _objective_9a82c_00000 | RUNNING  |       |     2.49816e-05 |                  2 |                            32 |
| _objective_9a82c_00001 | PENDING  |       |     3.92798e-05 |                  2 |                             8 |
| _objective_9a82c_00002 | PENDING  |       |     1.62407e-05 |                  5 |                     

(pid=359) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
(pid=359) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=359) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=359) Some weights of BertForSequenceClassification were not 

...

  1%|          | 3/311 [00:00<00:14, 21.19it/s]


(pid=629) {'train_runtime': 1089.2448, 'train_samples_per_second': 53.156, 'train_steps_per_second': 1.662, 'train_loss': 0.013621492148762909, 'epoch': 5.0}


100%|██████████| 311/311 [00:22<00:00, 13.54it/s]
2021-07-13 13:13:23,925	INFO tune.py:549 -- Total run time: 2362.05 seconds (2361.78 seconds for the tuning loop).


Result for _objective_9a82c_00002:
  date: 2021-07-13_13-13-23
  done: true
  epoch: 5.0
  eval_loss: 3.303964376449585
  eval_runtime: 22.9963
  eval_samples_per_second: 107.93
  eval_steps_per_second: 13.524
  experiment_id: d23fbf417b844e258528cd726108f8c9
  hostname: 28859bc61174
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 3.303964376449585
  pid: 629
  time_since_restore: 1119.5053822994232
  time_this_iter_s: 1119.5053822994232
  time_total_s: 1119.5053822994232
  timestamp: 1626182003
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9a82c_00002
  
== Status ==
Memory usage on this node: 4.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 GPU_group_0_d0ef30c3924031cf839503dcd57136c7, 0.0/1.0 CPU_group_d0ef30c3924031cf839503dcd57136c7, 0.0/1.0 accelerator_type:K80, 0.0/1.0 GPU_group_d0ef30c3924031cf839503dcd57136c7, 0.0/1.0 CPU_group_0_d0ef30c3924031cf839503dcd57

BestRun(run_id='9a82c_00000', objective=2.791937828063965, hyperparameters={'learning_rate': 2.49816047538945e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32})

Search Algorithm : 
  * If not provided `BasicVariantGenerator` Random search and grid search
  * Link : https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#tune-basicvariant 

### Best run 

Best_run : Optuna (n_trails = 5) with default hspace

```
BestRun(run_id='1', objective=0.8464898467063904, hyperparameters={'learning_rate': 3.2522034211592625e-06, 'num_train_epochs': 1, 'seed': 24, 'per_device_train_batch_size': 32})
```

Best_run : ray (n_trials = 3) with custom hspace
```
BestRun(run_id='9a82c_00000', objective=2.791937828063965, hyperparameters={'learning_rate': 2.49816047538945e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32})
```

## XL-Net

In [34]:
model = 'xlnet-base-cased'

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model,problem_type="multi_label_classification")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "problem_type": "multi_label_classification",
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summar

Under-sampling for 1000 due to large LM size 

In [36]:
train_df = train_df.sample(2000)
val_df = val_df.sample(2000)

In [37]:
train_dataset = ExplicitStereotypeDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [38]:
val_dataset = ExplicitStereotypeDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [39]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model, problem_type="multi_label_classification", num_labels = num_labels )

In [40]:
# Evaluate during training and a bit more often than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
# batch_size = 8

# training_args = TrainingArguments(
#     "test", evaluate_during_training=True, eval_steps=500, disable_tqdm=True)

trainer = Trainer(
    model_init= model_init,
    tokenizer = tokenizer,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

storing https://huggingface.co/xlnet-base-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987
creating metadata file for /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987
loading weights file https://huggingface.co/xlnet-base-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on 

In [41]:
# Defaut objective is the sum of all metrics when metrics are provided, so we have to maximize it.
best_run = trainer.hyperparameter_search(n_trials=5,hp_space=my_hp_space,backend="ray")

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
2021-07-22 16:57:59,849	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


== Status ==
Memory usage on this node: 3.4/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2021-07-22_16-58-04
Number of trials: 5/5 (5 PENDING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------|
| _objective_fb0b4_00000 | PENDING  |       |     2.49816e-05 |                  2 |                            32 |     15 |
| _objective_fb0b4_00001 | PENDING  |       |     4.11876e-05 |                  2 |                            16 |     39 |
| _objective_fb0b4_00002 | PENDING  |       |     1.62398e-05 |             

(pid=628) 2021-07-22 16:58:05.530782: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Memory usage on this node: 4.5/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2021-07-22_16-58-04
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------|
| _objective_fb0b4_00000 | RUNNING  |       |     2.49816e-05 |                  2 |                            32 |     15 |
| _objective_fb0b4_00001 | PENDING  |       |     4.11876e-05 |                  2 |                            16 |     39 |
| _objective_fb0b4_00002 | PENDING  |       |     1.62398e-05

(pid=628) Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
(pid=628) - This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=628) - This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=628) Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
(pid=628) You should probably TRAIN this model on a down-

...

Result for _objective_fb0b4_00003:
  date: 2021-07-22_17-06-32
  done: true
  epoch: 5.0
  eval_loss: 3.4166460037231445
  eval_runtime: 8.0051
  eval_samples_per_second: 249.84
  eval_steps_per_second: 31.23
  experiment_id: 6c62bf60174c4db795fcb059b94d9c33
  hostname: b2ef10663337
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 3.4166460037231445
  pid: 815
  time_since_restore: 180.0498378276825
  time_this_iter_s: 180.0498378276825
  time_total_s: 180.0498378276825
  timestamp: 1626973592
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: fb0b4_00003
  
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 GPU_group_0_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 CPU_group_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 accelerator_type:T4, 0.0/1.0 GPU_group_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 CPU_group_0_cfb6ad648905d2d9fd8c696e4f85467

(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=886) 2021-07-22 17:06:34.203949: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=886) Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
(pid=886) - This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=886) - This IS NOT expected if you are initializing XLNetForSequence

(pid=886) {'train_runtime': 74.1506, 'train_samples_per_second': 80.916, 'train_steps_per_second': 5.057, 'train_loss': 0.009768032073974609, 'epoch': 3.0}


100%|██████████| 250/250 [00:08<00:00, 30.62it/s]
2021-07-22 17:08:05,336	INFO tune.py:549 -- Total run time: 601.06 seconds (600.67 seconds for the tuning loop).


Result for _objective_fb0b4_00004:
  date: 2021-07-22_17-08-05
  done: true
  epoch: 3.0
  eval_loss: 2.861640691757202
  eval_runtime: 8.1852
  eval_samples_per_second: 244.343
  eval_steps_per_second: 30.543
  experiment_id: 879c77dce45f47f0bf7ff2b7eb2f5e4c
  hostname: b2ef10663337
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 2.861640691757202
  pid: 886
  time_since_restore: 89.31720423698425
  time_this_iter_s: 89.31720423698425
  time_total_s: 89.31720423698425
  timestamp: 1626973685
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: fb0b4_00004
  
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.3 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 CPU_group_0_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 accelerator_type:T4, 0.0/1.0 GPU_group_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 GPU_group_0_cfb6ad648905d2d9fd8c696e4f854679, 0.0/1.0 CPU_group_cfb6ad648905d2d9fd8c696e4f85467

 Best run : `LR : 2.21697e-05 |Epochs : 3 |Batch_size : 8 `
 Error with batch size of 32
  * Due to undersampling ?? 

### Best run 

XL-Net-base

In [42]:
best_run

BestRun(run_id='fb0b4_00000', objective=2.250964879989624, hyperparameters={'learning_rate': 2.49816047538945e-05, 'num_train_epochs': 2, 'seed': 15, 'per_device_train_batch_size': 32})

XL-Net-Large

In [ ]:
best_run

BestRun(run_id='84722_00003', objective=2.2981693744659424, hyperparameters={'learning_rate': 1.2323344486727979e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32})

## Roberta

In [22]:
model = 'roberta-base'

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model,problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [24]:
train_df = train_df.sample(2000)
val_df = val_df.sample(2000)

In [25]:
train_dataset = ExplicitStereotypeDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [26]:
val_dataset = ExplicitStereotypeDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [27]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model, problem_type="multi_label_classification", num_labels = num_labels )

In [28]:
# Evaluate during training and a bit more often than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
# batch_size = 8

training_args = TrainingArguments(
    "test", eval_steps=500, disable_tqdm=True)

trainer = Trainer(
    model_init= model_init,
    args = training_args,
    tokenizer = tokenizer,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
creating metadata file for /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', '

In [29]:
# Defaut objective is the sum of all metrics when metrics are provided, so we have to maximize it.
best_run = trainer.hyperparameter_search(n_trials=5, hp_space=my_hp_space, backend = 'ray' )

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
2021-07-25 09:15:45,548	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


== Status ==
Memory usage on this node: 3.4/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2021-07-25_09-15-49
Number of trials: 5/5 (5 PENDING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------|
| _objective_e7185_00000 | PENDING  |       |     2.49816e-05 |                  2 |                            32 |     15 |
| _objective_e7185_00001 | PENDING  |       |     4.11876e-05 |                  2 |                            16 |     39 |
| _objective_e7185_00002 | PENDING  |       |     1.62398e-05 |            

(pid=395) 2021-07-25 09:15:50.690869: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Memory usage on this node: 4.6/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/_objective_2021-07-25_09-15-49
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------|
| _objective_e7185_00000 | RUNNING  |       |     2.49816e-05 |                  2 |                            32 |     15 |
| _objective_e7185_00001 | PENDING  |       |     4.11876e-05 |                  2 |                            16 |     39 |
| _objective_e7185_00002 | PENDING  |       |     1.62398e-0

(pid=395) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
(pid=395) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=395) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=395) Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized

...

(pid=574) {'loss': 0.0203, 'learning_rate': 2.0426760281837013e-05, 'epoch': 2.0}
(pid=574) {'loss': 0.0009, 'learning_rate': 6.808920093945672e-06, 'epoch': 4.0}
(pid=574) {'train_runtime': 158.3546, 'train_samples_per_second': 63.149, 'train_steps_per_second': 7.894, 'train_loss': 0.008609517633914947, 'epoch': 5.0}
Result for _objective_e7185_00003:
  date: 2021-07-25_09-23-49
  done: true
  epoch: 5.0
  eval_loss: 0.0005985907046124339
  eval_runtime: 6.7897
  eval_samples_per_second: 294.566
  eval_steps_per_second: 36.821
  experiment_id: 6412b88a57e04490aae001ba3bccfeea
  hostname: a9d20365adc1
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.0005985907046124339
  pid: 574
  time_since_restore: 172.49245429039001
  time_this_iter_s: 172.49245429039001
  time_total_s: 172.49245429039001
  timestamp: 1627205029
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: e7185_00003
  
== Status ==
Memory usage on this node: 6.1/12.7 GiB
Using FIFO scheduling

(raylet) /usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(pid=641) 2021-07-25 09:23:51.155905: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
(pid=641) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias']
(pid=641) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another archite

(pid=641) {'train_runtime': 67.4374, 'train_samples_per_second': 88.971, 'train_steps_per_second': 5.561, 'train_loss': 0.01966500473022461, 'epoch': 3.0}


2021-07-25 09:25:14,086	INFO tune.py:549 -- Total run time: 564.52 seconds (564.17 seconds for the tuning loop).


Result for _objective_e7185_00004:
  date: 2021-07-25_09-25-13
  done: true
  epoch: 3.0
  eval_loss: 0.002161722630262375
  eval_runtime: 6.7136
  eval_samples_per_second: 297.903
  eval_steps_per_second: 37.238
  experiment_id: 327050abeb1e4f948275158db5974e75
  hostname: a9d20365adc1
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.002161722630262375
  pid: 641
  time_since_restore: 81.03456854820251
  time_this_iter_s: 81.03456854820251
  time_total_s: 81.03456854820251
  timestamp: 1627205113
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: e7185_00004
  
(pid=641) {'eval_loss': 0.002161722630262375, 'eval_runtime': 6.7136, 'eval_samples_per_second': 297.903, 'eval_steps_per_second': 37.238, 'epoch': 3.0}
== Status ==
Memory usage on this node: 6.0/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 GPU_group_154fee2e432168d0fcaf5f3577375dfa, 0.0/1.0 CPU_group_154fee2e

### Best run 

Roberta-base 

In [30]:
best_run 

BestRun(run_id='e7185_00003', objective=0.0005985907046124339, hyperparameters={'learning_rate': 3.404460046972836e-05, 'num_train_epochs': 5, 'seed': 22, 'per_device_train_batch_size': 8})

In [ ]:
best_run 

BestRun(run_id='48ed9_00003', objective=1.7053132057189941, hyperparameters={'learning_rate': 1.2323344486727979e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32})

## GPT-2

In [ ]:
model = 'gpt2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model,problem_type="multi_label_classification")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "problem_type": "multi_label_classification",
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": 

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# default to left padding
tokenizer.padding_side = "left"

Assigning [PAD] to the pad_token key of the tokenizer


In [ ]:
train_df = train_df.sample(2000)
val_df = val_df.sample(2000)

In [ ]:
train_dataset = ExplicitStereotypeDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [ ]:
val_dataset = ExplicitStereotypeDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_LEN
)

In [ ]:
from transformers import GPT2Config

# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(model, num_labels=num_labels)

Loading configuraiton...


loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
 

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model, problem_type="multi_label_classification")

In [ ]:
# Evaluate during training and a bit more often than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
# batch_size = 8

training_args = TrainingArguments(
    "test", label_names = LABEL_COLUMN, eval_steps=500, disable_tqdm=True)

trainer = Trainer(
    model_init= model_init,
    args = training_args,
    tokenizer = tokenizer,
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
)

In [ ]:
# Defaut objective is the sum of all metrics when metrics are provided, else minimize the loss 
best_run = trainer.hyperparameter_search(n_trials=5 )

### Best run 

In [ ]:
best_run 

# Class imbalance handling methods

Link 1 :
https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/

Link 2 : https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/

What?
  * Imbalance is most common problem
  * Class1 - 80 samples
  * Class2 - 20 samples 

Accuracy Paradox:
  * Accuracy metric may reflect the underlying class distribution.
    * Just predict class 1 irrespective of the input due to its class distribution.
    * Accuracy = `(80/100)*100 = 80%` 
    * But the model didnot learn anything.


Strategies:

1. Collect more data
2. Change performance metric:
  * Confusion matrix : Breaking the predictions into
    * Correct predictions:
      * True positive 
      * True Negative
    * Incorrect predictions:
      * False positive
      * False negative 
  * Precision : 
    * **correct positive prediction** out of **total positive predictions** (correct and incorrect).
  * Recall (sensitivity/TPR) : 
    * **Identified correct positive** predictions out of **total positive class in the dataset**.  
  * F1 score : 
    * Weighted average of precision and recall.
  * Kappa score:
    * Classification score normalized by the imbalance of classes in data.
    * Range from -1/0 - 1(perfect) 
  * ROC curve : 
    * TP (sensitivity) plotted against FP (1 – specificity) for each threshold used.
    * Useful for threshold selection 
      * Selecting threshold based on the dataset 
      * e.g.: Cancer screening : 
          * High FP along with TP is fine, as it is important to identify sufferers than having false negative.
    * ROC_AUC score : Gives performance of classifier over entire operating range.
    * Classifier comparison : Compare two models using ROC_AUC score. 
3. Resampling data 
  * Over-sampling:
      * Add copies from under-represented class.
      * Algorithms:
        * SMOTE(Synthetic minority over sampling technique)
          * Compute k-NN from minority class and impute.
        * Random over-sampling
      * Dis-advantage:
        * Impact generalization and may overfit the data.
  * Under-sampling:
    * Delete copies from over-represented class.
    * Algorithms
      * NearMiss
      * Random under-sampling
    * Dis-advantage:
      * May loose important information 
  * Points:
    * Consider testing random split and non-random (e.g. stratified) splits.
4. Different ML model:
  * Decision trees 
    * CART
    * Random forest
5. Penalized models:
  * Impose additional cost when predicting minority class to pay more attention.
    * Train model with class weights 
      * What are class weights ??
        * Different weights are given accordingly to the minority and majority classes which penalizes the misclassification during training according to the weights taking imbalance into consideration.
        * More weightage to minority and less to majority class.
        * In scikit learn when `class_weights = balanced`, the model assigns the **class weights inversely proportional to their respective frequencies**.
          `wj=n_samples / (n_classes * n_samplesj)`
        * Apply the weights to the weighted loss/cost function.
        * Results in the weighted loss (more error value to the minority and less error value to the majority class)
        * Correspondingly, the model coefficients/ hyper-parameters are adjusted w.r.t weighted loss.
    * Link : https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/
  * Focal loss for multi-class imbalanced data 
    * Link : https://www.dlology.com/blog/multi-class-classification-with-focal-loss-for-imbalanced-datasets/

6. Different problem
  * Anamoly detection
    * One-class classifier 
  * Change detection 


